In [8]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# On importe la fonction de notre nouveau fichier src/phase_folding.py
from src.phase_folding import create_folded_lightcurve

#Randomise the position of the planet inside the csv
from sklearn.utils import shuffle

# Configuration de TensorFlow (pour la métrique PR-AUC)
pr_auc = tf.keras.metrics.AUC(curve='PR', name='pr_auc')

print("Bibliothèques importées.")

Bibliothèques importées.


In [9]:
print("Chargement des données brutes...")
train_df = pd.read_csv('../data/raw/train.csv')

X_raw = train_df.drop('LABEL', axis=1).values
y = train_df['LABEL'].values - 1  # (0, 1)

print("Mélange des données...")
# On mélange X et y ensemble pour garder l'alignement
X_raw_shuffled, y_shuffled = shuffle(X_raw, y, random_state=42)

print(f"Forme de X_raw_shuffled : {X_raw_shuffled.shape}")
print(f"Forme de y_shuffled : {y_shuffled.shape}")

Chargement des données brutes...
Mélange des données...
Forme de X_raw_shuffled : (5087, 3197)
Forme de y_shuffled : (5087,)


In [10]:
N_BINS = 500
N_SAMPLE = 200  # On garde un petit échantillon pour tester

print(f"Démarrage du pré-traitement 'Phase Folding' (sur un ÉCHANTILLON mélangé de {N_SAMPLE} étoiles)...")

# On prend l'échantillon sur les données mélangées
X_sample = X_raw_shuffled[:N_SAMPLE]
y_sample = y_shuffled[:N_SAMPLE]

# Appliquer notre pipeline...
X_folded_sample = np.array([create_folded_lightcurve(row, n_bins=N_BINS) for row in X_sample])

print("Pré-traitement 'Phase Folding' sur échantillon terminé !")
print(f"Forme de X_folded_sample : {X_folded_sample.shape}")

# Vérifions la balance des classes (devrait être proche de 0.7%)
planets_in_sample = np.sum(y_sample)
print(f"Planètes dans l'échantillon : {planets_in_sample} sur {N_SAMPLE}")

Démarrage du pré-traitement 'Phase Folding' (sur un ÉCHANTILLON mélangé de 200 étoiles)...
Pré-traitement 'Phase Folding' sur échantillon terminé !
Forme de X_folded_sample : (200, 500)
Planètes dans l'échantillon : 4 sur 200


In [12]:
# On divise nos NOUVELLES données (X_folded)

print("Division des données (Train/Test) de l'ÉCHANTILLON...")
X_train, X_test, y_train, y_test = train_test_split(
    X_folded_sample,  # <-- Utiliser l'échantillon
    y_sample,         # <-- Utiliser l'échantillon
    test_size=0.2,
    random_state=42,
    stratify=y_sample # Stratifier sur l'échantillon
)

# Redimensionner pour le CNN (ajout du canal)
# La forme doit être (batch_size, steps, channels)
# (N, 500) -> (N, 500, 1)
X_train_cnn = np.expand_dims(X_train, axis=-1)
X_test_cnn = np.expand_dims(X_test, axis=-1)

# Notre nouvelle forme d'entrée pour le modèle
INPUT_SHAPE = (N_BINS, 1) # (500, 1)

print(f"Forme X_train_cnn : {X_train_cnn.shape}")
print(f"Forme X_test_cnn : {X_test_cnn.shape}")

Division des données (Train/Test) de l'ÉCHANTILLON...
Forme X_train_cnn : (160, 500, 1)
Forme X_test_cnn : (40, 500, 1)


In [13]:
# Nous pouvons réutiliser une architecture simple (comme la v5)
# mais l'input_shape doit être mise à jour.

def build_phase_model(input_shape):
    model = tf.keras.models.Sequential(name="CNN_v10_PhaseFolding")

    # Couche d'entrée
    model.add(tf.keras.layers.Input(shape=input_shape))

    # Blocs Convolutionnels
    model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=3))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=3))
    model.add(tf.keras.layers.Dropout(0.3))

    # Tête de classification
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # Sortie binaire

    return model

model = build_phase_model(INPUT_SHAPE)

# Compiler le modèle
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', pr_auc] # On suit notre métrique la plus importante
)

model.summary()

Model: "CNN_v10_PhaseFolding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 500, 16)        │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 166, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 166, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 166, 32)        │         2,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 55, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 55, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1760)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │        56,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,073 (230.75 KB)

 Trainable params: 59,073 (230.75 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# On veut sauvegarder le meilleur modèle basé sur la validation PR-AUC

# Nettoyer les anciens logs
!rmdir /S /Q .\\logs\\v10_phase_folding

# Callbacks
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir='./logs/v10_phase_folding', histogram_freq=1)

# On monitore 'val_pr_auc' en mode 'max'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='../models/cnn_v10_phase_folding_best.h5',
    save_best_only=True,
    monitor='val_pr_auc', # NOTRE MÉTRIQUE CLÉ
    mode='max',
    verbose=1
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_pr_auc',
    mode='max',
    patience=10, # Arrêter après 10 époques sans amélioration
    restore_best_weights=True,
    verbose=1
)

# Entraînement
print("Début de l'entraînement...")
history = model.fit(
    X_train_cnn,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_cnn, y_test),
    callbacks=[tensorboard_cb, checkpoint_cb, early_stopping_cb],
    verbose=1
)

Début de l'entraînement...
Epoch 1/100
1/5 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.7500 - loss: 0.5467 - pr_auc: 0.1511
Epoch 1: val_pr_auc improved from None to 0.03131, saving model to ../models/cnn_v10_phase_folding_best.h5


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.8562 - loss: 0.4799 - pr_auc: 0.0488 - val_accuracy: 0.9750 - val_loss: 0.4839 - val_pr_auc: 0.0313
Epoch 2/100
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9375 - loss: 0.4246 - pr_auc: 0.0211
Epoch 2: val_pr_auc did not improve from 0.03131
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8875 - loss: 0.4131 - pr_auc: 0.0132 - val_accuracy: 0.9750 - val_loss: 0.4266 - val_pr_auc: 0.0289
Epoch 3/100
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9062 - loss: 0.4515 - pr_auc: 0.0551
Epoch 3: val_pr_auc did not improve from 0.03131
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9312 - loss: 0.3681 - pr_auc: 0.0181 - val_accuracy: 0.9750 - val_loss: 0.3770 - val_pr_auc: 0.0250
Epoch 4/100
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9375 - loss: 0.3859 - pr_auc: 0.0165
Epoch 4: val_pr_auc did not improve from 0.03131
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9625 - loss: 0.3024 - pr_auc: 0.0137 - va